# Model training script

***

### Loading dataset and cleaning it

In [3]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from tqdm import tqdm
import warnings
import pprint
import pickle from pprint
warnings.filterwarnings("ignore")

df = pd.read_csv('transactions.csv')

all_columns = df.columns.tolist()
print("Columns in the dataset:", all_columns)


SyntaxError: invalid syntax (3802360479.py, line 16)

In [ ]:

# Creating a new feature 'prix_m2'
df['prix_m2'] = df['prix'] / df['surface_habitable']

# Filter for Paris data in 2022
df_idf = df[(df.departement == 75) & (df.date_transaction.str.startswith('2022-'))]

# Define features and target variable
X = df_idf[['code_postal', 
               'type_batiment','surface_habitable', 
              'latitude', 'longitude',]]
y = df_idf['prix_m2']

# Identify categorical and numerical columns
categorical_cols = [col for col in X.columns if X[col].dtype == 'object']
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

# Create transformers for numerical and categorical data
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Create a column transformer to apply transformations to the appropriate columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Create a pipeline that first transforms the data and then fits the model
model_pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Preprocess the features
X_processed = model_pipeline.fit_transform(X)
df_idf.head()


,id_transaction,date_transaction,prix,departement,id_ville,ville,code_postal,adresse,type_batiment,vefa,...,surface_habitable,id_parcelle_cadastre,latitude,longitude,surface_dependances,surface_locaux_industriels,surface_terrains_agricoles,surface_terrains_sols,surface_terrains_nature,prix_m2
6043985,10040024,2022-01-03,329910.0,75,112,PARIS 12,75012,260 AV DAUMESNIL,Appartement,False,...,30,75112000AX0008,48.836248,2.403751,{},{},{},{},{},10997.000000
6043986,10035062,2022-01-03,302400.0,75,102,PARIS 02,75002,52 RUE GRENETA,Appartement,False,...,26,75102000AM0139,48.865604,2.348148,{},{},{},{},{},11630.769231
6043987,10036657,2022-01-03,665000.0,75,117,PARIS 17,75017,26 BD DES BATIGNOLLES,Appartement,False,...,59,75117000CM0036,48.883309,2.324689,{0},{},{},{},{},11271.186441
6043988,10035700,2022-01-03,965000.0,75,105,PARIS 05,75005,31 RUE CENSIER,Appartement,False,...,63,75105000AU0078,48.839969,2.352576,"{0,0,0}",{},{},{},{},15317.460317
6043989,10029380,2022-01-03,200000.0,75,110,PARIS 10,75010,37 RUE DE PARADIS,Appartement,False,...,19,75110000AS0042,48.875213,2.350388,{},{},{},{},{},10526.315789


In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.8, random_state=42)

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

df_transactions = pd.read_csv('transactions.csv')

In [ ]:

if 'Unnamed: 0' in df_transactions.columns:
     df_transactions = df_transactions.drop('Unnamed: 0', axis=1)
    
df_transactions['prix_m2'] = df_transactions['prix'] / (df_transactions['surface_habitable'])

df_idf = df_transactions[(df_transactions.departement.isin([75, 77, 78, 91, 92, 93, 94, 95])) &
                         (df_transactions.date_transaction.str.startswith('2022')) &
                         (df_transactions.prix_m2 < 20000)]
surface_cols = [c for c in df_idf.columns if 'surface_' in c and c != 'surface_habitable']
for c in surface_cols:
     df_idf[c + '_sum'] = df_idf[c].apply(lambda x: sum(eval(x)) if 'NULL' not in x else 0)
df_idf = df_idf[df_idf[[c + '_sum' for c in surface_cols]].sum(axis=1) == 0]

df_paris = df_idf[(df_idf.departement == 75) &
                  (df_idf.prix_m2 > 3000)]

display (df_idf, df_paris),



,id_transaction,date_transaction,prix,departement,id_ville,ville,code_postal,adresse,type_batiment,vefa,...,surface_locaux_industriels,surface_terrains_agricoles,surface_terrains_sols,surface_terrains_nature,prix_m2,surface_dependances_sum,surface_locaux_industriels_sum,surface_terrains_agricoles_sum,surface_terrains_sols_sum,surface_terrains_nature_sum
6043985,10040024,2022-01-03,329910.0,75,112,PARIS 12,75012,260 AV DAUMESNIL,Appartement,False,...,{},{},{},{},10997.000000,0,0,0,0,0
6043986,10035062,2022-01-03,302400.0,75,102,PARIS 02,75002,52 RUE GRENETA,Appartement,False,...,{},{},{},{},11630.769231,0,0,0,0,0
6043987,10036657,2022-01-03,665000.0,75,117,PARIS 17,75017,26 BD DES BATIGNOLLES,Appartement,False,...,{},{},{},{},11271.186441,0,0,0,0,0
6043988,10035700,2022-01-03,965000.0,75,105,PARIS 05,75005,31 RUE CENSIER,Appartement,False,...,{},{},{},{},15317.460317,0,0,0,0,0
6043989,10029380,2022-01-03,200000.0,75,110,PARIS 10,75010,37 RUE DE PARADIS,Appartement,False,...,{},{},{},{},10526.315789,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8228478,13363983,2022-12-30,195000.0,95,127,CERGY,95000,2 RUE HAUTE DU TERTRE,Appartement,False,...,{},{},{},{},2407.407407,0,0,0,0,0
8228482,13367769,2022-12-30,268000.0,95,149,CHAUMONTEL,95270,2 RUE DE L ORME,Appartement,False,...,{},{},{},{},3941.176471,0,0,0,0,0
8228484,13363140,2022-12-30,129500.0,95,252,FRANCONVILLE LA GARENNE,95130,2 RUE DES FOLLES ENTREPRISES,Appartement,False,...,{},{},{},{},2443.396226,0,0,0,0,0
8228486,13367058,2022-12-30,635000.0,95,428,MONTMORENCY,95160,1 RUE DES SABLONS,Appartement,False,...,{},{},{},{},6546.391753,0,0,0,0,0


,id_transaction,date_transaction,prix,departement,id_ville,ville,code_postal,adresse,type_batiment,vefa,...,surface_locaux_industriels,surface_terrains_agricoles,surface_terrains_sols,surface_terrains_nature,prix_m2,surface_dependances_sum,surface_locaux_industriels_sum,surface_terrains_agricoles_sum,surface_terrains_sols_sum,surface_terrains_nature_sum
6043985,10040024,2022-01-03,329910.0,75,112,PARIS 12,75012,260 AV DAUMESNIL,Appartement,False,...,{},{},{},{},10997.000000,0,0,0,0,0
6043986,10035062,2022-01-03,302400.0,75,102,PARIS 02,75002,52 RUE GRENETA,Appartement,False,...,{},{},{},{},11630.769231,0,0,0,0,0
6043987,10036657,2022-01-03,665000.0,75,117,PARIS 17,75017,26 BD DES BATIGNOLLES,Appartement,False,...,{},{},{},{},11271.186441,0,0,0,0,0
6043988,10035700,2022-01-03,965000.0,75,105,PARIS 05,75005,31 RUE CENSIER,Appartement,False,...,{},{},{},{},15317.460317,0,0,0,0,0
6043989,10029380,2022-01-03,200000.0,75,110,PARIS 10,75010,37 RUE DE PARADIS,Appartement,False,...,{},{},{},{},10526.315789,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6079293,10050207,2022-12-30,460000.0,75,114,PARIS 14,75014,54 RUE DES PLANTES,Appartement,False,...,{},{},{},{},11794.871795,0,0,0,0,0
6079294,10057426,2022-12-30,228000.0,75,118,PARIS 18,75018,43 RUE DOUDEAUVILLE,Appartement,False,...,{},{},{},{},10857.142857,0,0,0,0,0
6079295,10057410,2022-12-30,1160000.0,75,109,PARIS 09,75009,28 RUE JEAN-BAPTISTE PIGALLE,Appartement,False,...,{},{},{},{},14500.000000,0,0,0,0,0
6079296,10055658,2022-12-30,840000.0,75,103,PARIS 03,75003,10 RUE DES FRANCS BOURGEOIS,Appartement,False,...,{},{},{},{},10769.230769,0,0,0,0,0


(None,)

### Data split

In [ ]:
# X = df_idf[[ 'longitude', 'latitude']].values
# y = df_idf['prix_m2'].values

# from sklearn.model_selection import train_test_split
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8)

### Import models + training

In [ ]:


"""'''params_grid = {
    'DTR': {
        'model': DecisionTreeRegressor(),
        'params': {
            'max_depth': (1, 101, 10), 
            'min_samples_split': (2, 21, 2) 
        }
    },
    'KNN': {
        'model': KNeighborsRegressor(),
        'params': {
            'n_neighbors': (1, 51, 5)  
        }
    },
    'LR': {
        'model': LinearRegression(),
        'params': {
            'fit_intercept': [True, False],
            'positive': [True, False]
        }
    },
    'RFR': {        'min_samples_leaf': (1, 12), 
            #'n_estimators': [100, 200, 300, 400, 500] 
        }
    }
}

for model_name, model_config in tqdm(params_grid.items()):
    gs = GridSearchCV(estimator=model_config['model'], param_grid=model_config['params'], n_jobs=-1)
        # gs = GridSearchCV(estimator=model_config['model'], param_grid=model_config['params'], n_jobs=-1)
    gs.fit(X_train, y_train)
    best_model = gs.best_estimator_
    best_params = gs.best_params_
    train_rmse = np.sqrt(mean_squared_error(y_train, best_model.predict(X_train)))
    test_rmse = np.sqrt(mean_squared_error(y_test, best_model.predict(X_test)))
    score = best_model.score(X_test, y_test)
    
    print(f"Model: {model_name}")
    print(f"Optimal params: {best_params}")
    print(f"Train RMSE: {train_rmse}")
    print(f"Test RMSE: {test_rmse}")
    print(f"Model Score: {score}")
    print() 
        'model': RandomForestRegressor(),
        'params': {
            'max_depth': (10, 110), 
    
"""

 25%|██▌       | 1/4 [00:04<00:14,  4.78s/it]

Model: DTR
Optimal params: {'max_depth': 1, 'min_samples_split': 2}
Train RMSE: 15845.205360874446
Test RMSE: 41351.066386107515
Model Score: -0.004491713358389715



 75%|███████▌  | 3/4 [00:09<00:02,  2.73s/it]

Model: KNN
Optimal params: {'n_neighbors': 51}
Train RMSE: 15688.626821711428
Test RMSE: 41290.19067122226
Model Score: -0.0015363293464745187

Model: LR
Optimal params: {'fit_intercept': False, 'positive': False}
Train RMSE: 15650.064732517156
Test RMSE: 41352.71028353767
Model Score: -0.004571581391481727



100%|██████████| 4/4 [01:05<00:00, 16.38s/it]

Model: RFR
Optimal params: {'max_depth': 10, 'min_samples_leaf': 12}
Train RMSE: 15298.688428888228
Test RMSE: 41298.447315541714
Model Score: -0.0019369163093021502



In [ ]:
from sklearn.ensemble import RandomForestRegressor
import pickle

optimal_rfr_model = RandomForestRegressor(max_depth=10, min_samples_leaf=12)
optimal_rfr_model.fit(X_train, y_train)

# Assurez-vous qu'il n'y a pas d'objet existant avec le nom 'optimal_rfr_model_idf.pkl'
pickle.dump(optimal_rfr_model, open('optimal_rfr_model_idf.pkl', 'wb'))
